In [ ]:
# TODO Bikin gen yang isinya random (99%)
# TODO Fitness/Cost Function (95%)
# TODO Generate Population (100%)
# TODO Selection (70%)
# TODO Crossover
# TODO Mutation
# TODO reassign population (eliminate weak gen/kromosom/individu)
# TODO Main Looping and Termination
# TODO Logging
# TODO Finalization
# TODO Develop Website
# TODO Deploy Website

In [58]:
import json
import random


input_file = 'input.json'
output_file = 'classes/output3.json'

# Baca data json 
def load_data(path):
    with open(path, 'r') as read_file:
        data = json.load(read_file)

    return data

# Masukan data json ke variabel
test = load_data(input_file)

In [59]:
# Test data, sudah masuk atau tidak
# print(test['Waktu']['Senin'])
print(test['List_Mapel'])

[{'Mapel': 'Pendidikan Agama dan Budi Pekerti', 'Alokasi_Jam': '3', 'ID': '1'}, {'Mapel': 'Pendidikan Pancasila dan Kewarganegaraan', 'ID': '2', 'Alokasi_Jam': '3'}, {'Mapel': 'Bahasa Indonesia', 'ID': '3', 'Alokasi_Jam': '3'}, {'Mapel': 'Bahasa Indonesia', 'ID': '4', 'Alokasi_Jam': '3'}, {'Mapel': 'Matematika', 'ID': '5', 'Alokasi_Jam': '3'}, {'Mapel': 'Matematika', 'ID': '6', 'Alokasi_Jam': '2'}, {'Mapel': 'Ilmu Pengetahuan Alam', 'ID': '7', 'Alokasi_Jam': '3'}, {'Mapel': 'Ilmu Pengetahuan Alam', 'ID': '8', 'Alokasi_Jam': '2'}, {'Mapel': 'Ilmu Pengetahuan Sosial', 'ID': '9', 'Alokasi_Jam': '2'}, {'Mapel': 'Ilmu Pengetahuan Sosial', 'ID': '10', 'Alokasi_Jam': '2'}, {'Mapel': 'Bahasa Inggris', 'ID': '11', 'Alokasi_Jam': '2'}, {'Mapel': 'Bahasa Inggris', 'ID': '12', 'Alokasi_Jam': '2'}, {'Mapel': 'Seni Budaya', 'ID': '13', 'Alokasi_Jam': '3'}, {'Mapel': 'Pendidikan Jasmani, Olah Raga, dan Kesehatan', 'ID': '14', 'Alokasi_Jam': '3'}, {'Mapel': 'Prakarya', 'ID': '15', 'Alokasi_Jam': '2'}]

In [60]:
# Fungsi mengambil satu buah elemen random dari sebuah list
def randomizer(source: list):
    temp = source[random.randrange(0, len(source))]
    return temp


# Test fungsi
keys_for_slicing = ["Mapel", "ID"]
rand_mapel = {key: randomizer(test['List_Mapel'])[key] for key in keys_for_slicing}
rand_guru = randomizer(test['Guru'])
print(rand_mapel)
print(rand_guru)


{'Mapel': 'Matematika', 'ID': '8'}
Candrakanta Tampubolon


In [ ]:
for angkatan in test['Kelas']:
    print(angkatan)


In [61]:
def generate_gen():
    # Generate sebuah gen/kromosom/individu, lalu simpan ke space
    space = {}

    for angkatan in test['Kelas']:
        kelas_temp = {}
        for kelas in test['Kelas'][angkatan]:
            hari_temp = {}
            for hari in test['Waktu']:
                alokasi_waktu = 0
                jam_temp = {}
                for jam in test['Waktu'][hari]:
                    if (alokasi_waktu == 0):
                        temp_mapel = randomizer(test['List_Mapel'])
                        temp_guru = randomizer(test['Guru'])
                        alokasi_waktu = int(temp_mapel['Alokasi_Jam'])
                    tempat_jam = {}
                    tempat_jam['Mapel'] = temp_mapel['Mapel']
                    tempat_jam['ID'] = temp_mapel['ID']
                    tempat_jam['Guru'] = temp_guru
                    jam_temp[jam] = tempat_jam
                    alokasi_waktu = alokasi_waktu - 1
                hari_temp[hari] = jam_temp
            kelas_temp[kelas] = hari_temp
        space[angkatan] = kelas_temp

    return space

In [ ]:
space = generate_gen()
space

In [ ]:
guru = {test['Guru'][i]: {'Mapel':[],'Slot_waktu':[]} for i in range(0, len(test['Guru']))}

print(guru)
slot_waktu = {}
for slot_jam in test['Waktu']['Senin']:
    slot_waktu[slot_jam] = []



In [62]:
# Cost Function
def cost_function(gen):
    cost = 0

    guru = {test['Guru'][i]: {'Mapel':[],'Slot_waktu':[]} for i in range(0, len(test['Guru']))}

    for angkatan in gen:
        for kelas in gen[angkatan]:
            for hari in gen[angkatan][kelas]:
                
                mapel_sehari = {}
                mapel_sehari['Mapel'] = []
                mapel_sehari['Total_Jam'] = []
                mapel_sehari['Mapel'].append(
                    gen[angkatan][kelas][hari]['7:00 - 7:40']['ID'])
                mapel_sehari['Total_Jam'].append(0)
                counter = 0

                counter_jam = 0
                for jam in gen[angkatan][kelas][hari]:
                    
                    # Koneksi guru ke mapel
                    if gen[angkatan][kelas][hari][jam]['ID'] not in guru[gen[angkatan][kelas][hari][jam]['Guru']]['Mapel']:
                        guru[gen[angkatan][kelas][hari][jam]['Guru']]['Mapel'].append(gen[angkatan][kelas][hari][jam]['ID'])

                    # Masukkan slot waktu tiap guru
                    guru[gen[angkatan][kelas][hari][jam]['Guru']]['Slot_waktu'].append(jam)
                    
                    # Mapel sehari ada apa aja ngitung total jam mapel
                    if mapel_sehari['Mapel'][counter] != gen[angkatan][kelas][hari][jam]['ID']:
                        mapel_sehari['Mapel'].append(gen[angkatan][kelas][hari][jam]['ID'])
                        mapel_sehari['Total_Jam'].append(1)
                        counter += 1
                    else:
                        mapel_sehari['Total_Jam'][counter] += 1
                        continue
                    
                # Jika ada mapel hanya satu jam pada satu hari (+1)
                if 1 in mapel_sehari['Total_Jam']:
                    cost += 1

                # Jika ada Mapel yang sama di satu hari (+1)
                result = dict((i, mapel_sehari['Mapel'].count(i)) for i in mapel_sehari['Mapel'])
                if len(result) < len(mapel_sehari['Mapel']):
                    cost += 1

    for j in guru:
        
        # Jika ada Guru yang sama di slot waktu yang sama disetiap hari (irisan) (+1)
        result = dict((i, guru[j]['Slot_waktu'].count(i)) for i in guru[j]['Slot_waktu'])
        cost += len(guru[j]['Slot_waktu']) - len(result)
        
        # Jika Guru Memegang lebih dari 4 mapel (+1)
        if len(guru[j]['Mapel']) > 4:
            cost += 1

    # TODO
    # Jika ada mapel yang sama di satu angkatan tapi beda guru (+1)
    
    return cost


In [63]:
# Generate Population
def generate_population(num_gen: int):
    population = []
    for i in range(0, num_gen):
        gen = {}
        # space = generate_gen()
        gen['Timetable'] = generate_gen()
        gen['Cost'] = cost_function(gen['Timetable'])
        population.append(gen)
    return population

# print(max(hasil))
# print(population)


In [173]:
# Selection
temp = generate_population(4)

temp2 = {}
# temp
for i in range(0, len(temp)):
    temp2[i] = temp[i]['Cost']
    # print(temp[i]['Cost'])

# min(temp2, key=d.get)
print(temp2)
index = min(temp2, key=temp2.get)
print(index)
parent1 = temp[index]
# print(parent1)
temp.pop(index)

temp2 = {}
# temp
for i in range(0, len(temp)):
    temp2[i] = temp[i]['Cost']
    # print(temp[i]['Cost'])
print(temp2)
index = min(temp2, key=temp2.get)
print(index)
parent2 = temp[index]

print(parent1['Cost'])
print(parent2['Cost'])


{0: 951, 1: 926, 2: 914, 3: 934}
2
{0: 951, 1: 926, 2: 934}
1
914
926


In [ ]:
print(cost_function(space))


In [ ]:
cost

In [ ]:

# Test modifikasi data pada sebuah gen/kromosom/individu

print(space['VII']['A']['Senin']['7:00 - 7:40'])

space['VII']['A']['Senin']['7:00 - 7:40']['Guru'] = 'Yayat'
space['VII']['A']['Senin']['7:00 - 7:40']['Mapel'] = 'PKN'
print(space['VII']['A']['Senin']['7:00 - 7:40'])

# mapel indo
{0, 1, 2}



In [ ]:
# Fungsi untuk menulis hasil solusi/gen/kromosom/individu ke file
def write_data(data, path):
    with open(path, 'w') as write_file:
        json.dump(data, write_file, indent=4)

In [ ]:
write_data(generate_population(4), 'output_random.json')


In [ ]:
# Testing only
hasil = []
for i in range(0,2000):
    space = generate_gen()
    hasil.append(cost_function(space))

print(max(hasil))



In [ ]:
# Testing only
for i in range(0,2000):
    space = generate_gen()
    guru = {test['Guru'][i]: {'Mapel':[],'Slot_waktu':[]} for i in range(0, len(test['Guru']))}

    slot_waktu = {}
    for slot_jam in test['Waktu']['Senin']:
        slot_waktu[slot_jam] = []

    # Cost Function
    cost = 0

    for angkatan in space:
        for kelas in space[angkatan]:
            for hari in space[angkatan][kelas]:
                
                mapel_sehari = {}
                mapel_sehari['Mapel'] = []
                mapel_sehari['Total_Jam'] = []
                mapel_sehari['Mapel'].append(
                    space[angkatan][kelas][hari]['7:00 - 7:40']['ID'])
                mapel_sehari['Total_Jam'].append(0)
                counter = 0

                counter_jam = 0
                for jam in space[angkatan][kelas][hari]:
                    # slot_waktu[jam].append(space[angkatan][kelas][hari][jam]['Guru'])
                    
                    # Koneksi guru ke mapel
                    if space[angkatan][kelas][hari][jam]['ID'] not in guru[space[angkatan][kelas][hari][jam]['Guru']]['Mapel']:
                        guru[space[angkatan][kelas][hari][jam]['Guru']]['Mapel'].append(space[angkatan][kelas][hari][jam]['ID'])
                    
                    # print(jam)
                    guru[space[angkatan][kelas][hari][jam]['Guru']]['Slot_waktu'].append(jam)
                    
                    # Mapel sehari ada apa aja ngitung total jam mapel
                    if mapel_sehari['Mapel'][counter] != space[angkatan][kelas][hari][jam]['ID']:
                        mapel_sehari['Mapel'].append(space[angkatan][kelas][hari][jam]['ID'])
                        mapel_sehari['Total_Jam'].append(1)
                        counter += 1
                    else:
                        mapel_sehari['Total_Jam'][counter] += 1
                        continue
                    
                # Jika ada mapel hanya satu jam pada satu hari (+1)
                # print(guru)
                # print(mapel_sehari)
                if 1 in mapel_sehari['Total_Jam']:
                    cost += 1

                # Jika ada Mapel yang sama di satu hari (+1)
                result = dict((i, mapel_sehari['Mapel'].count(i)) for i in mapel_sehari['Mapel'])
                # print(result)
                if len(result) < len(mapel_sehari['Mapel']):
                    cost += 1

    # Jika Guru Memegang lebih dari 4 mapel (+1)
    # print(guru)
    # print(guru['Sakura Pertiwi']['Slot_waktu'])
    for j in guru:
        # print(j)
        # print(guru[j]['Slot_waktu'].count('7:00 - 7:40'))
        result = dict((i, guru[j]['Slot_waktu'].count(i)) for i in guru[j]['Slot_waktu'])
        # print(result)
        cost += len(guru[j]['Slot_waktu']) - len(result)
        # if len(result) < len(guru[j]['Slot_waktu']):
        #     cost +


        if len(guru[j]['Mapel']) > 4:
            cost += 1
        # print(guru[i])

    # Jika ada Guru yang sama di slot waktu yang sama disetiap hari (irisan) (+1)
    # for j in slot_waktu:
    #     result = dict((i, slot_waktu[j].count(i)) for i in slot_waktu[j])
    #     # print(result)
    #     if len(result) < len(slot_waktu[j]):
    #         cost += 1
    #         # print("test")
    # Jika ada mapel yang sama di satu angkatan tapi beda guru (+1)

    print(cost)
